In [2]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 

In [3]:
days = export_from_RISKCUSTOM(f"""select distinct "reportDate"
from "RISKACCESS"."unifiedOcp" 
where "sourceTable" in ('unifiedDealsBrazil_externalPurchasesRecap','unifiedDealsBrazil_externalSalesOrders')
order by "reportDate"
""")
days.head(5)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


,reportDate
0,2024-09-01
1,2024-09-02
2,2024-09-03
3,2024-09-04
4,2024-09-05


In [4]:
query = f"""select * 
from "RISKACCESS"."unifiedOcp" 
where "sourceTable" in ('unifiedDealsBrazil_externalPurchasesRecap','unifiedDealsBrazil_externalSalesOrders')
and "reportDate" = TO_DATE('{days.iloc[0,:].astype(str).item()}', 'YYYY-MM-DD')
--fetch first 10 rows only"""
data_OCP_export = export_from_RISKCUSTOM(query)
data_OCP_export.shape

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


(9282, 40)

In [5]:
# effectiveCurrency,volumeEffectiveCurrency
data_OCP_export.groupby(as_index=False, dropna=False, by=['salesUnitsId','sourceTable'])['volumeUsd'].agg('sum')

,salesUnitsId,sourceTable,volumeUsd
0,AECOR,unifiedDealsBrazil_externalPurchasesRecap,1.669245e+08
1,AETRD,unifiedDealsBrazil_externalPurchasesRecap,3.215486e+06
2,BRFTO,unifiedDealsBrazil_externalPurchasesRecap,-5.497500e+08
3,BRFTO,unifiedDealsBrazil_externalSalesOrders,9.333603e+14
4,BRSFT,unifiedDealsBrazil_externalPurchasesRecap,8.374083e+06


In [12]:
# FTO
# Pur
pur_str = '''AN 0
AS 208
MAP 87
MOP G 209
NK 13
NP 40
NPK MOP 102
SSP 116
TSP 60
UREA GRAN 214
OTHER 14'''
# Sales
sales_str = """AN -13
AS -228
CAN -5
MAP -139
MOP G -348
NK -25
NP -7
NPK MOP -46
NPK OTHER -11
NPS -25
SSP -113
TSP -48
UREA GRAN -217
OTHER -22
"""
prices_str = """MOP G	12 	$283 
UREA GRAN	-62 	$375 
AN	8 	$205 
AS	22 	$183 
CAN	2 	$246 
MAP	-42 	$612 
SSP	50 	$187 
TSP	27 	$440 
NP	35 	$564 
PHOSPHATE ROCK	2 	$137 
NPK 21:1:21 [MOP] COMP	-3 	$337 
NPK 19:4:19 [MOP] COMP	1 	$297 
NPK 15:15:15 [MOP] COMP	2 	$394 
NPK 16:16:16 [MOP] COMP	8 	$358 
NPS	1 	$468 
CN WS (19CA) GRAN	-0 	$403 
OTHER	40 	$462"""


In [14]:
data_pur = pd.DataFrame([x.rsplit(' ', 1) for x in pur_str.split('\n')]).rename(columns={0:'BRFTO',1:'Purchases'})
data_sales = pd.DataFrame([x.rsplit(' ', 1) for x in sales_str.split('\n')]).rename(columns={0:'BRFTO',1:'Sales'})
data_merge_BRFTO = data_sales.merge(data_pur, how='outer', on='BRFTO').loc[1:,:]

In [16]:
# pd.DataFrame().rename(columns={0:'BRFTO',1:'Purchases'})
data_prices = pd.DataFrame([x.split('\t') for x in prices_str.split('\n')]).rename(columns={0:'BRFTO',2:'Prices'}).drop(columns=1)
data_prices.Prices = data_prices.Prices.str[1:4]
data_merge = data_prices.merge(data_merge_BRFTO, how='outer', on='BRFTO')

In [27]:
# data_merge = data_merge.fillna(0)
data_merge[['Prices','Sales','Purchases']] = data_merge[['Prices','Sales','Purchases']].astype(float)
data_merge['amount_Purchases'] = data_merge.Prices * data_merge.Purchases
data_merge['amount_Sales'] = data_merge.Prices * data_merge.Sales
data_merge.sum()

BRFTO               ANASCANCN WS (19CA) GRANMAPMOP GNKNPNPK 15:15:...
Prices                                                         5951.0
Sales                                                         -1247.0
Purchases                                                      1063.0
amount_Purchases                                             307825.0
amount_Sales                                                -378609.0
dtype: object

In [29]:
query = f"""select distinct "sourceTable"
from "RISKACCESS"."unifiedOcp" 
"""
data_sourceTable = export_from_RISKCUSTOM(query)
data_sourceTable

,sourceTable
0,flowDealsReportChanges
1,unifiedDealsMarketSuek
2,unifiedDealsBrazil_sapSalesOrders
3,unifiedDealsMarketSuek_inventory
4,unifiedDealsBrazil_inventory_sapSalesOrders
5,unifiedDealsBrazil_inventory_externalPurchases...
6,unifiedDealsBrazil_purchasesRecapBRHRG
7,unifiedDealsBrazil_inventory_externalSalesOrders
8,flowDealsReportChanges_inventory
9,bankAccountsBalance


In [ ]:
# unifiedDealsBrazil_purchasesRecapBRHRG
# unifiedDealsBrazil_externalPurchasesRecap
# unifiedDealsBrazil_externalSalesOrders
# unifiedDealsBrazil_sapSalesOrders
# flowDealsReportChanges
# unifiedDealsBrazil_inventory_purchasesRecapBRHRG